# Cutom Outlier Treatment Algo
this is custom outlier treatment using IsolationForest

In [694]:
import numpy as np
import pandas as pd
import pandasql as ps
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

In [695]:
def filter_list(lst):
    for idx in range(len(lst)):
        if lst[idx] == -1:
            lst[idx] = 1
        else:
            break

    return np.array(lst)

filter_list([-1 ,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,-1,-1,-1])

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1,
       -1])

In [696]:
#Read CSVFile
df = pd.read_csv("dataset for outlier treatment.csv")
final_df = pd.DataFrame(columns=['dateid','StoreId','StoreName','StoreBillingSoftwareName','RegionId','RegionName','final_brandname','TotalStockSoldQuantity','TotalStockSoldAmount_x','TotalStockSoldAmount_y'])
df

,dateid,StoreId,StoreName,StoreBillingSoftwareName,RegionId,RegionName,final_brandname,TotalStockSoldQuantity,TotalStockSoldAmount
0,20200101,3220,A.M.Assorted,Marg,56,Howrah,GLUCORYL-MV,3.0,318.60
1,20200101,488,A.N.Pharma,NewMedica,5,Thane,TAXIM-O,252.0,18975.78
2,20200101,488,A.N.Pharma,NewMedica,5,Thane,SWICH-CV,4.0,771.08
3,20200101,488,A.N.Pharma,NewMedica,5,Thane,ZOCEF-CV,15.0,3107.10
4,20200101,488,A.N.Pharma,NewMedica,5,Thane,GLUCORYL-MV,2.0,207.86
5,20200101,488,A.N.Pharma,NewMedica,5,Thane,PAN-D,5.0,635.70
6,20200101,488,A.N.Pharma,NewMedica,5,Thane,GLUCORYL-M,16.0,1452.10
7,20200101,488,A.N.Pharma,NewMedica,5,Thane,EUCLIDE-M,16.0,771.36
8,20200101,488,A.N.Pharma,NewMedica,5,Thane,PAN,1815.0,63808.17
9,20200101,488,A.N.Pharma,NewMedica,5,Thane,EMTY,27.0,2311.20


In [1]:
regions = [3,4,5,15,55,56,66]
for region in regions: 
    print(region)
    query = """
                SELECT distinct StoreId FROM df where RegionId = '"""+str(region)+"""' ORDER BY StoreId,final_brandname
             """
    stores_df = ps.sqldf(query, locals())
    #print(stores_df)
    cnt =0
    for index, row in stores_df.iterrows():
        query = """
                    SELECT distinct final_brandname FROM df where RegionId = '"""+str(region)+"""' and StoreId = '"""+str(stores_df.at[index,'StoreId'])+"""' ORDER BY final_brandname
                 """
        brands_df = ps.sqldf(query, locals())
        for index1, row1 in brands_df.iterrows():
            #print("brand",str(brands_df.at[index1,'final_brandname']))
            query = """
                        SELECT * FROM df where RegionId = '"""+str(region)+"""' and StoreId = '"""+str(stores_df.at[index,'StoreId'])+"""' and final_brandname = '"""+str(brands_df.at[index1,'final_brandname'])+"""' ORDER BY final_brandname
                     """
            my_df = ps.sqldf(query, locals())

            #print(my_df)
            result = outliertreatment(my_df)
            #print(result)
            if result is not None:
                mergedDf = my_df.merge(result, left_index=True, right_index=True)
                #print(mergedDf)
                final_df = final_df.append(mergedDf,ignore_index=True)
            else:
                my_df['TotalStockSoldAmount_y'] = ""
                mergedDf = my_df.merge(my_df, left_index=True, right_index=True)
                #print(mergedDf)
                final_df = final_df.append(mergedDf,ignore_index=True)
                

In [698]:
final_df.to_csv('/home/juned/PythonWork/OutlierPandas/dataset for outlier treatment-withoutlier.csv', header=True, index=False)

In [692]:
def outliertreatment(my_df):
    #print(my_df)
    ndf = pd.DataFrame(my_df['TotalStockSoldAmount'])
    ndf = ndf.sort_values(by=['TotalStockSoldAmount'])#.reset_index(drop=True)
    #print("::::::::::::::::::::ndf:::::::::::::::::::::",ndf)
    df_less100000 = ndf[ndf['TotalStockSoldAmount'] < 100000]
    #print(df_less100000)
    if len(df_less100000)==0 and len(ndf)>0:
        # This is a Specialcase where everything above 100000 means everything is outlier, in this case replace outlier with that brand of entire regions mean
        RegionId = my_df['StoreId'][0]
        BrandName = my_df['final_brandname'][0]
        #Read CSVFile
        df = pd.read_csv("dataset for outlier treatment.csv")
        #df
        query = """
                    SELECT * FROM df where RegionId = '"""+str(RegionId)+"""' and final_brandname = '"""+str(BrandName)+"""' and (TotalStockSoldAmount >0 and TotalStockSoldAmount <100000) 
                 """
        my_df1 = ps.sqldf(query, locals())
        ndf1 = pd.DataFrame(my_df1['TotalStockSoldAmount'])
        ndf1 = ndf1.sort_values(by=['TotalStockSoldAmount']).reset_index(drop=True)
        df_less100000 = ndf1[ndf1['TotalStockSoldAmount'] < 100000]
        
        #Algo
        if len(df_less100000>0):
            rng = np.random.RandomState(20)
            clf = IsolationForest( behaviour = 'new', max_samples='auto', random_state = rng, contamination= 'auto')
            preds = clf.fit_predict(df_less100000)
            preds = filter_list(preds)
            #Find outlier
            df_outlier = df_less100000['TotalStockSoldAmount'][preds == -1]
            
            #Find Mean of non-outlier values
            mean = np.mean(df_less100000["TotalStockSoldAmount"][preds == 1])
            
            ndf["TotalStockSoldAmount"][ndf['TotalStockSoldAmount'] >= 100000] = mean
            #print(ndf)
            return ndf
        else:
            return ndf

    else:
        #Algo
        rng = np.random.RandomState(20)
        clf = IsolationForest( behaviour = 'new', max_samples='auto', random_state = rng, contamination= 'auto')
        preds = clf.fit_predict(df_less100000)
        preds = filter_list(preds)
        #print(preds)
        

        #Find outlier
        df_outlier = df_less100000['TotalStockSoldAmount'][preds == -1]
        
        #Find Mean of non-outlier values
        mean = np.mean(df_less100000["TotalStockSoldAmount"][preds == 1])
        
        #Replace Mean value with outlier values
        df_less100000["TotalStockSoldAmount"][preds == -1] = mean
        
        #Replace by index new outlier value in main df
        ndf.loc[df_less100000.index] = df_less100000
        
        #Noew its time to Replace >100000 values with mean value
        pd.options.mode.chained_assignment = None  # default='warn'
        df_greater100000 = ndf[ndf['TotalStockSoldAmount'] >= 100000]
        if len(df_greater100000)>0:
            df_greater100000["TotalStockSoldAmount"][df_greater100000['TotalStockSoldAmount'] >= 100000] = mean
            #print("::::::df_greater100000 value replaced by mean:::::::")
            # Replace >100000 values by index new outlier value in main df 
            ndf.loc[df_greater100000.index] = df_greater100000
            
        return ndf

In [375]:
# np.random.seed(1)
rng = np.random.RandomState(20)
clf = IsolationForest( behaviour = 'new', max_samples='auto', random_state = rng, contamination= 'auto')
preds = clf.fit_predict(df_less100000)
preds

array([-1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

In [376]:
df_outlier = df_less100000['TotalStockSoldAmount'][preds == -1]
df_outlier

0         42.25
1         42.25
2         42.25
3         42.25
4         42.25
5         42.25
6         42.25
7         42.25
8         42.44
22        84.50
23        84.50
24        84.50
25        84.50
26        84.50
531     3399.35
532     3442.32
533     3460.08
534     3504.65
535     3510.35
536     3510.35
537     3540.97
538     3662.79
539     3758.06
540     3782.61
541     3849.58
542     3861.20
543     3996.47
544     4011.60
545     4025.81
546     4065.98
         ...   
692    15213.85
693    15220.95
694    15246.83
695    15335.45
696    15491.68
697    15571.96
698    15671.62
699    16384.16
700    16923.02
701    17079.17
702    17610.76
703    17723.27
704    18309.16
705    18799.57
706    19036.34
707    19561.15
708    19868.38
709    20174.97
710    21926.58
711    24434.43
712    25323.66
713    25426.88
714    25846.50
715    27940.37
716    28753.51
717    30228.23
718    33062.72
719    33882.11
720    35399.15
721    36252.88
Name: TotalStockSoldAmou

In [378]:
mid = np.mean(df_less100000["TotalStockSoldAmount"][preds == 1])
mid

972.1301353965189

In [341]:
df_less100000["TotalStockSoldAmount"][preds == -1] = mid
df_less100000

,TotalStockSoldAmount
0,408.58
1,817.16
2,2042.90
3,2451.48
4,1430.03
5,1430.03


In [343]:
ndf.loc[df_less100000.index] = df_less100000
ndf

,TotalStockSoldAmount
0,408.58
1,817.16
2,2042.90
3,2451.48
4,1430.03
5,1430.03
6,200909.98
7,249233.80
8,299717.82
9,316509.50


In [345]:
# df_greater100000 = ndf[ndf['TotalStockSoldAmount'] >= 100000]
df_greater100000["TotalStockSoldAmount"][df_greater100000['TotalStockSoldAmount'] >= 100000] = mid
df_greater100000

,TotalStockSoldAmount
6,1430.03
7,1430.03
8,1430.03
9,1430.03
10,1430.03
11,1430.03
12,1430.03
13,1430.03
14,1430.03
15,1430.03


In [348]:
ndf.loc[df_greater100000.index] = df_greater100000
ndf

,TotalStockSoldAmount
0,408.58
1,817.16
2,2042.90
3,2451.48
4,1430.03
5,1430.03
6,1430.03
7,1430.03
8,1430.03
9,1430.03
